In [30]:
import pymystem3 as mystem
stemmer = mystem.Mystem()
from joblib import dump, load
from catboost import CatBoostRegressor

import nltk
nltk.download('punkt')
stopwords = nltk.corpus.stopwords.words("russian")

from sklearn.ensemble import RandomForestRegressor
\
import csv
from collections import defaultdict
from natasha import (
    NamesExtractor,
    DatesExtractor,
    MoneyExtractor,
)
import re

[nltk_data] Downloading package punkt to /home/annikura/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [2]:
def to_tokens(s):
    return [token['text'].lower() for token in stemmer.analyze(s)]

def read_csv(filename, has_answs=False, sep=','):
    pid = []
    qid = []
    texts = []
    questions = []
    answers = []

    with open(filename, 'r') as file:
        qa = csv.reader(file, delimiter=sep)
        for line in qa:
            pid.append(line[0])
            qid.append(line[1])
            texts.append(to_tokens(line[2]))
            questions.append(to_tokens(line[3]))
            if has_answs:
                answers.append(to_tokens(line[4]))  
    pid = pid[1:]
    qid = qid[1:]
    texts = [text[:-1] for text in texts][1:]
    questions = [question[:-1] for question in questions][1:]
    answers = [answer[:-1] for answer in answers][1:]
    return pid, qid, texts, questions, answers

In [3]:
pids, qids, texts, questions, answers = read_csv('train_qa.csv', True)

In [4]:
i = 1
print(texts[i], questions[i], answers[i], sep='\n')

['телескоп', ' ', 'имеет', ' ', 'модульную', ' ', 'структуру', ' ', 'и', ' ', 'содержит', ' ', 'пять', ' ', 'отсеков', ' ', 'для', ' ', 'оптических', ' ', 'приборов', '. ', 'один', ' ', 'из', ' ', 'отсеков', ' ', 'в', ' ', 'течение', ' ', 'долгого', ' ', 'времени', ' (', '1993', '—', '2009', ' ', 'годы', ') ', 'занимала', ' ', 'корректирующая', ' ', 'оптическая', ' ', 'система', ' (', 'costar', '), ', 'установленная', ' ', 'во', ' ', 'время', ' ', 'первой', ' ', 'экспедиции', ' ', 'обслуживания', ' ', 'в', ' ', '1993', ' ', 'году', ' ', 'для', ' ', 'компенсации', ' ', 'неточности', ' ', 'изготовления', ' ', 'главного', ' ', 'зеркала', '. ', 'поскольку', ' ', 'все', ' ', 'приборы', ', ', 'установленные', ' ', 'после', ' ', 'запуска', ' ', 'телескопа', ', ', 'имеют', ' ', 'встроенные', ' ', 'системы', ' ', 'коррекции', ' ', 'дефекта', ', ', 'во', ' ', 'время', ' ', 'последней', ' ', 'экспедиции', ' ', 'стало', ' ', 'возможно', ' ', 'демонтировать', ' ', 'систему', ' ', 'costar', ' ', 'и'

In [5]:
def filter_text(text):
    return [word for word in text if word.isalpha() or word.isnumeric()]

tmp_question_categories = defaultdict(int)
question_categories = defaultdict(int)

for question in questions:
    question = filter_text(question)
    for i in range(1, 3):
        tmp_question_categories[' '.join(question[:i])] += 1
tmp_categories = list(map(lambda x: x[0], filter(lambda x: x[1] > 50, tmp_question_categories.items())))

for question in questions:
    question = filter_text(question)
    for i in range(3, 0, -1):
        if ' '.join(question[:i]) in tmp_categories:
            question_categories[' '.join(question[:i])] += 1
            break
question_categories = list(map(lambda x: x[0], filter(lambda x: x[1] > 50, question_categories.items())))
question_categories = {x: i for i, x in enumerate(question_categories)}
question_categories

{'где': 0,
 'как называется': 1,
 'какая': 2,
 'чему': 3,
 'на какие': 4,
 'на что': 5,
 'что': 6,
 'что в': 7,
 'сколько лет': 8,
 'какую': 9,
 'куда': 10,
 'до какого': 11,
 'кто': 12,
 'в каком': 13,
 'в': 14,
 'когда': 15,
 'какой': 16,
 'в какой': 17,
 'как': 18,
 'какие': 19,
 'по каким': 20,
 'за что': 21,
 'какова': 22,
 'какие виды': 23,
 'от какого': 24,
 'какими': 25,
 'под': 26,
 'чего': 27,
 'на чем': 28,
 'кем был': 29,
 'для чего': 30,
 'как называются': 31,
 'какое': 32,
 'чем': 33,
 'с каким': 34,
 'сколько': 35,
 'что произошло': 36,
 'что представляют': 37,
 'какой город': 38,
 'что по': 39,
 'с какого': 40,
 'кому': 41,
 'почему': 42,
 'как называлась': 43,
 'о чем': 44,
 'что является': 45,
 'по': 46,
 'что стало': 47,
 'с какими': 48,
 'каким': 49,
 'с кем': 50,
 'к': 51,
 'к чему': 52,
 'на': 53,
 'кем': 54,
 'к какой': 55,
 'для': 56,
 'кто был': 57,
 'при': 58,
 'что не': 59,
 'во сколько': 60,
 'с': 61,
 'с чем': 62,
 'кем была': 63,
 'когда была': 64,
 'каков

In [34]:
def get_scores(one, other):
    one_set = set(one)
    other_set = set(other)
    intersection = len(one_set.intersection(other))
    return 0 if not one else intersection / len(one), 0 if not other else intersection / len(other)
    
def f1(one, other):
    rec, prec = get_scores(one, other)
    if not (rec + prec):
        return 0
    return 2 * (rec * prec) / (rec + prec)

def get_lemma(x):
    if 'analysis' in x and x['analysis']:
        return x['analysis'][0].get('lex', x['text'])
    return x['text']

##############################################################################

def filter_text(text):
    return [word for word in text if word.isalpha() or word.isnumeric()]

def contains_year(text):
    for token in text:
        if len(token) == 4 and token.isnumeric():
            return True
    return False

def filter_stopwords(text):
    return [word for word in text if word not in stopwords]


##############################################################################

def f1_no_stowords(one, other):
    return f1(filter_stopwords(one), filter_stopwords(other))

def first_words_match(one, other):
    return int(one[0] == other[0])

##############################################################################

def question_category(question):
    for i in range(3, 0, -1):
        s = ''.join(question[:i])
        if s in question_categories:
            return question_categories[s]
    return len(question_categories)


##############################################################################

def is_prefix(pref, here):
    return pref == here[:len(pref)]

def contains_name(tokens):
    for token in tokens:
        if 'analysis' in token:
            a = token['analysis'] 
            if a and ('фам' in a[0]['gr'] or 'имя' in a[0]['gr']):
                return 1
    return 0

def countpar(tokens, pref_parts, in_parts):
    pref_ans = [0] * len(pref_parts)
    in_ans = [0] * len(in_parts)
    for token in tokens:
        if 'analysis' in token:
            a = token['analysis'] 
            if len(a) > 0:
                for i, part in enumerate(pref_parts):
                    if is_prefix(part, a[0]['gr']):
                        pref_ans[i] += 1

                for i, part in enumerate(in_parts):
                    if part in a[0]['gr']:
                        in_ans[i] += 1
    return pref_ans + in_ans


##############################################################################

def build_textual_features(tokens, analyzed):
    return [
        len(tokens),
        contains_name(analyzed),
        contains_year(tokens),
        *countpar(analyzed,
                  ["S", "CONJ", "V", "ADV", "A", "NUM"],
                  ["жен", "муж", "сред", "гео", "им,", "род", "дат", "вин", "твор", "пр"]),
        
    ]  

def build_crosstextual_features(tokens, t_analyzed, question, q_lemmas):
    return [
        *build_textual_features(tokens, t_analyzed),
        len(tokens) / len(question),
        f1(tokens, question),
        f1([get_lemma(x) for x in t_analyzed], q_lemmas),
        f1_no_stowords(tokens, question),
        *get_scores(tokens, question),
    ]

def isok(x):
    return x.isalpha() or x.isnumeric()

def build_features(lines, answers=None, local_num=8, left_num=5, right_num=5):
    punkt_re = re.compile(";|!|\\?|\\.{3}")
    cnt = 0
    stats = 0
    total = 0
    
    dataset = []
    candidates = []
    ys = []
    
    for pid, qid, text_tokens, question_tokens in lines:
        print('Text {} out of {}. Candidates: {}'.format(cnt, len(lines), len(dataset)))

        question_tokens = filter_text(question_tokens)
        question_text = ' '.join(question_tokens)
        question_analyzed = list(filter(lambda x: isok(x['text']), stemmer.analyze(question_text)))
        question_lemmas = [get_lemma(x) for x in question_analyzed]
        
        question_features = [
            *build_textual_features(question_tokens, question_analyzed),
            qid,
            question_category(question_tokens),
        ]

        if answers:
            answer_tokens = filter_text(answers[cnt])

        sentences = [to_tokens(x) for x in nltk.sent_tokenize(''.join(text_tokens), language="russian")]
        scores = [(i, get_scores(question_tokens, filter_text(sen))[0]) for i, sen in enumerate(sentences)]
        
        sen_id = sorted(scores, key=lambda x: x[1], reverse=True)[0][0]
        total += 1
        if answers:
            if ' '.join(answer_tokens) in ' '.join(filter_text(sentences[sen_id])):
                stats += 1

        for sentence in [sentences[sen_id]]:
            right_s = defaultdict(lambda: None)
            left_tokens = [] 
            left_analyzed = []

            analyzed = stemmer.analyze(''.join(sentence))
            for i in range(len(sentence)):
                left_tokens_features = build_crosstextual_features(left_tokens, left_analyzed, question_tokens, question_lemmas)
                if not isok(sentence[i]):
                    continue

                local_tokens = [] 
                local_analyzed = []
                for j in range(20):
                    if i + j >= len(sentence) or len(local_tokens) == local_num:
                        break
                    token = sentence[i + j]
                    if punkt_re.search(token):
                        break
                    if not isok(token):
                        continue
                    local_tokens.append(token)
                    local_analyzed.append(analyzed[i + j])

                    if len(local_tokens) < 2:
                        continue
                        
                    if answers:
                        ys.append(f1(local_tokens, answer_tokens))
                        if ys[-1] == 0:
                            if (i + j) % 2 or (i // j + j) % 2:
                                ys.pop()
                                continue
                    
                    e = len(local_tokens) - 1
                    while e > 0:
                        if 'analysis' not in local_analyzed[e]:
                            break
                        if not local_analyzed[e]['analysis']:
                            break
                        gr = local_analyzed[e]['analysis'][0]['gr']
                        if 'CONJ' in gr or 'PR' in gr or 'INTJ' in gr or gr[0] == 'V':
                            e -= 1
                        else:
                            break
                        
                    candidates.append(' '.join(local_tokens[:e + 1]))
                                
                    if not right_s[j + 1]:
                        right_tokens = []
                        right_analyzed = []
                        for k in range(20):
                            if i + j + k + 1 >= len(sentence) or len(right_tokens) == right_num:
                                break;
                            if isok(sentence[i + j + 1 + k]):
                                right_tokens.append(sentence[i + j + 1 + k])
                                right_analyzed.append(analyzed[i + j + 1 + k])
                        right_s[j + 1] = build_crosstextual_features(right_tokens, right_analyzed, question_tokens, question_lemmas)
                        
                    right_features = right_s[j + 1]
                    
                    features = [
                        pid,
                        i / len(sentence),
                        (len(sentence) - i - j) / len(sentence),
                        len(sentence),
                        *question_features,
                        *build_crosstextual_features(local_tokens, local_analyzed, question_tokens, question_lemmas),
                        *left_tokens_features,
                        *right_features,
                    ]
                    dataset.append(features)
                if isok(sentence[i]):
                    left_tokens.append(sentence[i])
                    left_analyzed.append(analyzed[i])
                    if len(left_tokens) > left_num:
                        left_tokens.pop(0)
                        left_analyzed.pop(0)
        cnt += 1
    print(stats / total)
    return candidates, dataset, ys

In [7]:
len(answers)

50364

In [8]:
sz = 20000

li = list(zip(pids, qids, texts, questions))[:sz]
phrases, data, ys = build_features(li, answers)

Text 0 out of 20000. Candidates: 0
Text 1 out of 20000. Candidates: 36
Text 2 out of 20000. Candidates: 153
Text 3 out of 20000. Candidates: 241
Text 4 out of 20000. Candidates: 327
Text 5 out of 20000. Candidates: 370
Text 6 out of 20000. Candidates: 507
Text 7 out of 20000. Candidates: 736
Text 8 out of 20000. Candidates: 776
Text 9 out of 20000. Candidates: 918
Text 10 out of 20000. Candidates: 1071
Text 11 out of 20000. Candidates: 1103
Text 12 out of 20000. Candidates: 1136
Text 13 out of 20000. Candidates: 1144
Text 14 out of 20000. Candidates: 1225
Text 15 out of 20000. Candidates: 1309
Text 16 out of 20000. Candidates: 1318
Text 17 out of 20000. Candidates: 1348
Text 18 out of 20000. Candidates: 1491
Text 19 out of 20000. Candidates: 1631
Text 20 out of 20000. Candidates: 1703
Text 21 out of 20000. Candidates: 1834
Text 22 out of 20000. Candidates: 1884
Text 23 out of 20000. Candidates: 2031
Text 24 out of 20000. Candidates: 2132
Text 25 out of 20000. Candidates: 2244
Text 26 o

Text 219 out of 20000. Candidates: 21377
Text 220 out of 20000. Candidates: 21600
Text 221 out of 20000. Candidates: 21664
Text 222 out of 20000. Candidates: 21695
Text 223 out of 20000. Candidates: 21866
Text 224 out of 20000. Candidates: 21952
Text 225 out of 20000. Candidates: 22079
Text 226 out of 20000. Candidates: 22169
Text 227 out of 20000. Candidates: 22218
Text 228 out of 20000. Candidates: 22262
Text 229 out of 20000. Candidates: 22359
Text 230 out of 20000. Candidates: 22451
Text 231 out of 20000. Candidates: 22554
Text 232 out of 20000. Candidates: 22730
Text 233 out of 20000. Candidates: 22766
Text 234 out of 20000. Candidates: 22830
Text 235 out of 20000. Candidates: 22877
Text 236 out of 20000. Candidates: 23002
Text 237 out of 20000. Candidates: 23031
Text 238 out of 20000. Candidates: 23200
Text 239 out of 20000. Candidates: 23240
Text 240 out of 20000. Candidates: 23318
Text 241 out of 20000. Candidates: 23409
Text 242 out of 20000. Candidates: 23644
Text 243 out of 

Text 433 out of 20000. Candidates: 40090
Text 434 out of 20000. Candidates: 40120
Text 435 out of 20000. Candidates: 40171
Text 436 out of 20000. Candidates: 40212
Text 437 out of 20000. Candidates: 40296
Text 438 out of 20000. Candidates: 40404
Text 439 out of 20000. Candidates: 40464
Text 440 out of 20000. Candidates: 40509
Text 441 out of 20000. Candidates: 40743
Text 442 out of 20000. Candidates: 40864
Text 443 out of 20000. Candidates: 40966
Text 444 out of 20000. Candidates: 41093
Text 445 out of 20000. Candidates: 41181
Text 446 out of 20000. Candidates: 41254
Text 447 out of 20000. Candidates: 41489
Text 448 out of 20000. Candidates: 41578
Text 449 out of 20000. Candidates: 41632
Text 450 out of 20000. Candidates: 41663
Text 451 out of 20000. Candidates: 41740
Text 452 out of 20000. Candidates: 41897
Text 453 out of 20000. Candidates: 42097
Text 454 out of 20000. Candidates: 42140
Text 455 out of 20000. Candidates: 42195
Text 456 out of 20000. Candidates: 42283
Text 457 out of 

Text 642 out of 20000. Candidates: 59783
Text 643 out of 20000. Candidates: 59876
Text 644 out of 20000. Candidates: 59992
Text 645 out of 20000. Candidates: 60100
Text 646 out of 20000. Candidates: 60318
Text 647 out of 20000. Candidates: 60418
Text 648 out of 20000. Candidates: 60708
Text 649 out of 20000. Candidates: 60807
Text 650 out of 20000. Candidates: 60982
Text 651 out of 20000. Candidates: 61022
Text 652 out of 20000. Candidates: 61072
Text 653 out of 20000. Candidates: 61185
Text 654 out of 20000. Candidates: 61241
Text 655 out of 20000. Candidates: 61330
Text 656 out of 20000. Candidates: 61396
Text 657 out of 20000. Candidates: 61494
Text 658 out of 20000. Candidates: 61539
Text 659 out of 20000. Candidates: 61550
Text 660 out of 20000. Candidates: 61713
Text 661 out of 20000. Candidates: 61730
Text 662 out of 20000. Candidates: 61941
Text 663 out of 20000. Candidates: 62002
Text 664 out of 20000. Candidates: 62067
Text 665 out of 20000. Candidates: 62194
Text 666 out of 

Text 854 out of 20000. Candidates: 80747
Text 855 out of 20000. Candidates: 80868
Text 856 out of 20000. Candidates: 80893
Text 857 out of 20000. Candidates: 80950
Text 858 out of 20000. Candidates: 80974
Text 859 out of 20000. Candidates: 81122
Text 860 out of 20000. Candidates: 81335
Text 861 out of 20000. Candidates: 81414
Text 862 out of 20000. Candidates: 81490
Text 863 out of 20000. Candidates: 81566
Text 864 out of 20000. Candidates: 81615
Text 865 out of 20000. Candidates: 81703
Text 866 out of 20000. Candidates: 81731
Text 867 out of 20000. Candidates: 81779
Text 868 out of 20000. Candidates: 81902
Text 869 out of 20000. Candidates: 81966
Text 870 out of 20000. Candidates: 82024
Text 871 out of 20000. Candidates: 82078
Text 872 out of 20000. Candidates: 82154
Text 873 out of 20000. Candidates: 82182
Text 874 out of 20000. Candidates: 82349
Text 875 out of 20000. Candidates: 82405
Text 876 out of 20000. Candidates: 82483
Text 877 out of 20000. Candidates: 82535
Text 878 out of 

...

Text 19133 out of 20000. Candidates: 1797104
Text 19134 out of 20000. Candidates: 1797142
Text 19135 out of 20000. Candidates: 1797239
Text 19136 out of 20000. Candidates: 1797349
Text 19137 out of 20000. Candidates: 1797415
Text 19138 out of 20000. Candidates: 1797479
Text 19139 out of 20000. Candidates: 1797585
Text 19140 out of 20000. Candidates: 1797634
Text 19141 out of 20000. Candidates: 1797801
Text 19142 out of 20000. Candidates: 1797830
Text 19143 out of 20000. Candidates: 1798044
Text 19144 out of 20000. Candidates: 1798231
Text 19145 out of 20000. Candidates: 1798464
Text 19146 out of 20000. Candidates: 1798574
Text 19147 out of 20000. Candidates: 1798674
Text 19148 out of 20000. Candidates: 1798770
Text 19149 out of 20000. Candidates: 1798824
Text 19150 out of 20000. Candidates: 1798903
Text 19151 out of 20000. Candidates: 1799074
Text 19152 out of 20000. Candidates: 1799259
Text 19153 out of 20000. Candidates: 1799353
Text 19154 out of 20000. Candidates: 1799424
Text 19155

Text 19317 out of 20000. Candidates: 1814854
Text 19318 out of 20000. Candidates: 1814982
Text 19319 out of 20000. Candidates: 1815023
Text 19320 out of 20000. Candidates: 1815159
Text 19321 out of 20000. Candidates: 1815284
Text 19322 out of 20000. Candidates: 1815342
Text 19323 out of 20000. Candidates: 1815413
Text 19324 out of 20000. Candidates: 1815463
Text 19325 out of 20000. Candidates: 1815492
Text 19326 out of 20000. Candidates: 1815556
Text 19327 out of 20000. Candidates: 1815762
Text 19328 out of 20000. Candidates: 1815827
Text 19329 out of 20000. Candidates: 1816025
Text 19330 out of 20000. Candidates: 1816100
Text 19331 out of 20000. Candidates: 1816181
Text 19332 out of 20000. Candidates: 1816313
Text 19333 out of 20000. Candidates: 1816399
Text 19334 out of 20000. Candidates: 1816446
Text 19335 out of 20000. Candidates: 1816531
Text 19336 out of 20000. Candidates: 1816581
Text 19337 out of 20000. Candidates: 1816673
Text 19338 out of 20000. Candidates: 1816835
Text 19339

Text 19506 out of 20000. Candidates: 1832667
Text 19507 out of 20000. Candidates: 1832807
Text 19508 out of 20000. Candidates: 1832865
Text 19509 out of 20000. Candidates: 1832939
Text 19510 out of 20000. Candidates: 1833012
Text 19511 out of 20000. Candidates: 1833086
Text 19512 out of 20000. Candidates: 1833111
Text 19513 out of 20000. Candidates: 1833249
Text 19514 out of 20000. Candidates: 1833463
Text 19515 out of 20000. Candidates: 1833538
Text 19516 out of 20000. Candidates: 1833690
Text 19517 out of 20000. Candidates: 1833788
Text 19518 out of 20000. Candidates: 1833950
Text 19519 out of 20000. Candidates: 1833993
Text 19520 out of 20000. Candidates: 1834112
Text 19521 out of 20000. Candidates: 1834150
Text 19522 out of 20000. Candidates: 1834301
Text 19523 out of 20000. Candidates: 1834344
Text 19524 out of 20000. Candidates: 1834395
Text 19525 out of 20000. Candidates: 1834518
Text 19526 out of 20000. Candidates: 1834584
Text 19527 out of 20000. Candidates: 1834804
Text 19528

Text 19696 out of 20000. Candidates: 1850960
Text 19697 out of 20000. Candidates: 1851097
Text 19698 out of 20000. Candidates: 1851389
Text 19699 out of 20000. Candidates: 1851429
Text 19700 out of 20000. Candidates: 1851434
Text 19701 out of 20000. Candidates: 1851511
Text 19702 out of 20000. Candidates: 1851584
Text 19703 out of 20000. Candidates: 1851633
Text 19704 out of 20000. Candidates: 1851827
Text 19705 out of 20000. Candidates: 1851870
Text 19706 out of 20000. Candidates: 1851938
Text 19707 out of 20000. Candidates: 1851990
Text 19708 out of 20000. Candidates: 1852134
Text 19709 out of 20000. Candidates: 1852149
Text 19710 out of 20000. Candidates: 1852190
Text 19711 out of 20000. Candidates: 1852237
Text 19712 out of 20000. Candidates: 1852511
Text 19713 out of 20000. Candidates: 1852668
Text 19714 out of 20000. Candidates: 1852770
Text 19715 out of 20000. Candidates: 1852864
Text 19716 out of 20000. Candidates: 1852912
Text 19717 out of 20000. Candidates: 1852980
Text 19718

Text 19887 out of 20000. Candidates: 1868329
Text 19888 out of 20000. Candidates: 1868459
Text 19889 out of 20000. Candidates: 1868555
Text 19890 out of 20000. Candidates: 1868661
Text 19891 out of 20000. Candidates: 1868805
Text 19892 out of 20000. Candidates: 1868857
Text 19893 out of 20000. Candidates: 1868894
Text 19894 out of 20000. Candidates: 1868952
Text 19895 out of 20000. Candidates: 1869036
Text 19896 out of 20000. Candidates: 1869121
Text 19897 out of 20000. Candidates: 1869173
Text 19898 out of 20000. Candidates: 1869230
Text 19899 out of 20000. Candidates: 1869311
Text 19900 out of 20000. Candidates: 1869439
Text 19901 out of 20000. Candidates: 1869477
Text 19902 out of 20000. Candidates: 1869515
Text 19903 out of 20000. Candidates: 1869577
Text 19904 out of 20000. Candidates: 1869667
Text 19905 out of 20000. Candidates: 1869740
Text 19906 out of 20000. Candidates: 1869916
Text 19907 out of 20000. Candidates: 1870041
Text 19908 out of 20000. Candidates: 1870158
Text 19909

In [9]:
print(data[:5], ys[:5])

[['14754', 0.0, 0.9047619047619048, 21, 8, 0, False, 3, 0, 1, 1, 3, 0, 0, 1, 4, 0, 3, 3, 0, 4, 0, 2, '60544', 0, 2, 0, False, 1, 0, 0, 0, 1, 0, 0, 0, 1, 0, 2, 1, 0, 2, 0, 0, 0.25, 0.4, 0.4, 0.5, 1.0, 0.25, 0, 0, False, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0.0, 0, 0, 0, 0, 0.0, 5, 0, False, 2, 0, 1, 0, 1, 0, 0, 1, 3, 0, 1, 2, 0, 2, 0, 2, 0.625, 0.7692307692307693, 0.7692307692307693, 0.8, 1.0, 0.625], ['14754', 0.0, 0.8095238095238095, 21, 8, 0, False, 3, 0, 1, 1, 3, 0, 0, 1, 4, 0, 3, 3, 0, 4, 0, 2, '60544', 0, 3, 0, False, 1, 0, 0, 0, 1, 0, 0, 0, 1, 0, 2, 1, 0, 2, 0, 0, 0.375, 0.5454545454545454, 0.5454545454545454, 0.5, 1.0, 0.375, 0, 0, False, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0.0, 0, 0, 0, 0, 0.0, 5, 0, False, 2, 0, 1, 0, 1, 0, 0, 1, 3, 0, 1, 2, 0, 2, 0, 2, 0.625, 0.6153846153846154, 0.6153846153846154, 0.8, 0.8, 0.5], ['14754', 0.0, 0.7142857142857143, 21, 8, 0, False, 3, 0, 1, 1, 3, 0, 0, 1, 4, 0, 3, 3, 0, 4, 0, 2, '60544', 0, 4, 0, False, 2, 0, 0, 0, 1, 0

In [10]:
model = CatBoostRegressor(iterations=10000,
                          learning_rate=0.01,
                          depth=12 )
# Fit model
model.fit(data[:sz], ys[:sz])

0:	learn: 0.2659273	total: 224ms	remaining: 37m 24s
1:	learn: 0.2648011	total: 316ms	remaining: 26m 17s
2:	learn: 0.2635961	total: 410ms	remaining: 22m 47s
3:	learn: 0.2626362	total: 518ms	remaining: 21m 33s
4:	learn: 0.2616465	total: 613ms	remaining: 20m 25s
5:	learn: 0.2607072	total: 705ms	remaining: 19m 33s
6:	learn: 0.2597329	total: 824ms	remaining: 19m 35s
7:	learn: 0.2587997	total: 918ms	remaining: 19m 6s
8:	learn: 0.2578006	total: 1.01s	remaining: 18m 40s
9:	learn: 0.2567837	total: 1.13s	remaining: 18m 47s
10:	learn: 0.2557991	total: 1.22s	remaining: 18m 28s
11:	learn: 0.2549399	total: 1.31s	remaining: 18m 12s
12:	learn: 0.2540177	total: 1.42s	remaining: 18m 12s
13:	learn: 0.2529825	total: 1.51s	remaining: 18m 1s
14:	learn: 0.2520173	total: 1.61s	remaining: 17m 50s
15:	learn: 0.2511155	total: 1.72s	remaining: 17m 52s
16:	learn: 0.2502214	total: 1.82s	remaining: 17m 49s
17:	learn: 0.2492829	total: 1.92s	remaining: 17m 42s
18:	learn: 0.2483583	total: 2.03s	remaining: 17m 46s
19:	l

155:	learn: 0.1713059	total: 15.5s	remaining: 16m 18s
156:	learn: 0.1708592	total: 15.6s	remaining: 16m 18s
157:	learn: 0.1705320	total: 15.7s	remaining: 16m 18s
158:	learn: 0.1701241	total: 15.8s	remaining: 16m 18s
159:	learn: 0.1697828	total: 15.9s	remaining: 16m 18s
160:	learn: 0.1694667	total: 16s	remaining: 16m 17s
161:	learn: 0.1691801	total: 16.1s	remaining: 16m 18s
162:	learn: 0.1688338	total: 16.2s	remaining: 16m 17s
163:	learn: 0.1684980	total: 16.3s	remaining: 16m 17s
164:	learn: 0.1682049	total: 16.4s	remaining: 16m 18s
165:	learn: 0.1678224	total: 16.5s	remaining: 16m 18s
166:	learn: 0.1675175	total: 16.6s	remaining: 16m 17s
167:	learn: 0.1672395	total: 16.7s	remaining: 16m 18s
168:	learn: 0.1668194	total: 16.8s	remaining: 16m 18s
169:	learn: 0.1665369	total: 16.9s	remaining: 16m 18s
170:	learn: 0.1662029	total: 17s	remaining: 16m 18s
171:	learn: 0.1658681	total: 17.1s	remaining: 16m 19s
172:	learn: 0.1655303	total: 17.2s	remaining: 16m 19s
173:	learn: 0.1651824	total: 17.

308:	learn: 0.1302696	total: 30.9s	remaining: 16m 9s
309:	learn: 0.1300542	total: 31s	remaining: 16m 9s
310:	learn: 0.1297651	total: 31.1s	remaining: 16m 8s
311:	learn: 0.1294926	total: 31.2s	remaining: 16m 8s
312:	learn: 0.1292914	total: 31.3s	remaining: 16m 8s
313:	learn: 0.1291267	total: 31.4s	remaining: 16m 8s
314:	learn: 0.1289631	total: 31.5s	remaining: 16m 8s
315:	learn: 0.1287353	total: 31.6s	remaining: 16m 8s
316:	learn: 0.1284876	total: 31.7s	remaining: 16m 7s
317:	learn: 0.1282332	total: 31.8s	remaining: 16m 8s
318:	learn: 0.1280488	total: 31.9s	remaining: 16m 7s
319:	learn: 0.1277574	total: 32s	remaining: 16m 7s
320:	learn: 0.1275104	total: 32.1s	remaining: 16m 7s
321:	learn: 0.1273424	total: 32.2s	remaining: 16m 7s
322:	learn: 0.1271714	total: 32.3s	remaining: 16m 7s
323:	learn: 0.1269405	total: 32.4s	remaining: 16m 7s
324:	learn: 0.1268025	total: 32.5s	remaining: 16m 7s
325:	learn: 0.1266072	total: 32.6s	remaining: 16m 6s
326:	learn: 0.1264686	total: 32.7s	remaining: 16m 

464:	learn: 0.1047726	total: 46.4s	remaining: 15m 50s
465:	learn: 0.1046914	total: 46.5s	remaining: 15m 50s
466:	learn: 0.1045218	total: 46.6s	remaining: 15m 50s
467:	learn: 0.1043841	total: 46.7s	remaining: 15m 50s
468:	learn: 0.1042437	total: 46.8s	remaining: 15m 50s
469:	learn: 0.1041197	total: 46.9s	remaining: 15m 50s
470:	learn: 0.1040108	total: 47s	remaining: 15m 50s
471:	learn: 0.1038829	total: 47.1s	remaining: 15m 49s
472:	learn: 0.1038047	total: 47.1s	remaining: 15m 49s
473:	learn: 0.1036762	total: 47.3s	remaining: 15m 49s
474:	learn: 0.1035481	total: 47.4s	remaining: 15m 49s
475:	learn: 0.1034229	total: 47.4s	remaining: 15m 49s
476:	learn: 0.1032609	total: 47.5s	remaining: 15m 49s
477:	learn: 0.1031756	total: 47.6s	remaining: 15m 48s
478:	learn: 0.1029798	total: 47.7s	remaining: 15m 48s
479:	learn: 0.1028673	total: 47.8s	remaining: 15m 48s
480:	learn: 0.1027479	total: 47.9s	remaining: 15m 48s
481:	learn: 0.1026017	total: 48s	remaining: 15m 48s
482:	learn: 0.1024686	total: 48.

619:	learn: 0.0886825	total: 1m 1s	remaining: 15m 33s
620:	learn: 0.0886401	total: 1m 1s	remaining: 15m 33s
621:	learn: 0.0885365	total: 1m 1s	remaining: 15m 33s
622:	learn: 0.0884403	total: 1m 1s	remaining: 15m 33s
623:	learn: 0.0883944	total: 1m 2s	remaining: 15m 32s
624:	learn: 0.0883373	total: 1m 2s	remaining: 15m 32s
625:	learn: 0.0882610	total: 1m 2s	remaining: 15m 32s
626:	learn: 0.0881457	total: 1m 2s	remaining: 15m 32s
627:	learn: 0.0880956	total: 1m 2s	remaining: 15m 32s
628:	learn: 0.0880246	total: 1m 2s	remaining: 15m 32s
629:	learn: 0.0879381	total: 1m 2s	remaining: 15m 31s
630:	learn: 0.0878531	total: 1m 2s	remaining: 15m 31s
631:	learn: 0.0877993	total: 1m 2s	remaining: 15m 31s
632:	learn: 0.0877313	total: 1m 2s	remaining: 15m 31s
633:	learn: 0.0876517	total: 1m 3s	remaining: 15m 31s
634:	learn: 0.0875828	total: 1m 3s	remaining: 15m 31s
635:	learn: 0.0875118	total: 1m 3s	remaining: 15m 31s
636:	learn: 0.0874159	total: 1m 3s	remaining: 15m 31s
637:	learn: 0.0873332	total:

...

9285:	learn: 0.0113964	total: 15m 46s	remaining: 1m 12s
9286:	learn: 0.0113950	total: 15m 46s	remaining: 1m 12s
9287:	learn: 0.0113936	total: 15m 47s	remaining: 1m 12s
9288:	learn: 0.0113911	total: 15m 47s	remaining: 1m 12s
9289:	learn: 0.0113900	total: 15m 47s	remaining: 1m 12s
9290:	learn: 0.0113875	total: 15m 47s	remaining: 1m 12s
9291:	learn: 0.0113858	total: 15m 47s	remaining: 1m 12s
9292:	learn: 0.0113848	total: 15m 47s	remaining: 1m 12s
9293:	learn: 0.0113841	total: 15m 47s	remaining: 1m 11s
9294:	learn: 0.0113820	total: 15m 47s	remaining: 1m 11s
9295:	learn: 0.0113802	total: 15m 47s	remaining: 1m 11s
9296:	learn: 0.0113791	total: 15m 47s	remaining: 1m 11s
9297:	learn: 0.0113780	total: 15m 48s	remaining: 1m 11s
9298:	learn: 0.0113761	total: 15m 48s	remaining: 1m 11s
9299:	learn: 0.0113757	total: 15m 48s	remaining: 1m 11s
9300:	learn: 0.0113747	total: 15m 48s	remaining: 1m 11s
9301:	learn: 0.0113727	total: 15m 48s	remaining: 1m 11s
9302:	learn: 0.0113703	total: 15m 48s	remaining:

9436:	learn: 0.0111783	total: 16m 2s	remaining: 57.4s
9437:	learn: 0.0111767	total: 16m 2s	remaining: 57.3s
9438:	learn: 0.0111748	total: 16m 2s	remaining: 57.2s
9439:	learn: 0.0111741	total: 16m 2s	remaining: 57.1s
9440:	learn: 0.0111728	total: 16m 2s	remaining: 57s
9441:	learn: 0.0111712	total: 16m 2s	remaining: 56.9s
9442:	learn: 0.0111693	total: 16m 2s	remaining: 56.8s
9443:	learn: 0.0111683	total: 16m 2s	remaining: 56.7s
9444:	learn: 0.0111678	total: 16m 3s	remaining: 56.6s
9445:	learn: 0.0111662	total: 16m 3s	remaining: 56.5s
9446:	learn: 0.0111652	total: 16m 3s	remaining: 56.4s
9447:	learn: 0.0111640	total: 16m 3s	remaining: 56.3s
9448:	learn: 0.0111623	total: 16m 3s	remaining: 56.2s
9449:	learn: 0.0111605	total: 16m 3s	remaining: 56.1s
9450:	learn: 0.0111581	total: 16m 3s	remaining: 56s
9451:	learn: 0.0111565	total: 16m 3s	remaining: 55.9s
9452:	learn: 0.0111555	total: 16m 3s	remaining: 55.8s
9453:	learn: 0.0111536	total: 16m 3s	remaining: 55.7s
9454:	learn: 0.0111517	total: 16

9588:	learn: 0.0109605	total: 16m 17s	remaining: 41.9s
9589:	learn: 0.0109586	total: 16m 17s	remaining: 41.8s
9590:	learn: 0.0109574	total: 16m 18s	remaining: 41.7s
9591:	learn: 0.0109560	total: 16m 18s	remaining: 41.6s
9592:	learn: 0.0109545	total: 16m 18s	remaining: 41.5s
9593:	learn: 0.0109531	total: 16m 18s	remaining: 41.4s
9594:	learn: 0.0109516	total: 16m 18s	remaining: 41.3s
9595:	learn: 0.0109505	total: 16m 18s	remaining: 41.2s
9596:	learn: 0.0109495	total: 16m 18s	remaining: 41.1s
9597:	learn: 0.0109486	total: 16m 18s	remaining: 41s
9598:	learn: 0.0109467	total: 16m 18s	remaining: 40.9s
9599:	learn: 0.0109447	total: 16m 18s	remaining: 40.8s
9600:	learn: 0.0109430	total: 16m 19s	remaining: 40.7s
9601:	learn: 0.0109419	total: 16m 19s	remaining: 40.6s
9602:	learn: 0.0109398	total: 16m 19s	remaining: 40.5s
9603:	learn: 0.0109385	total: 16m 19s	remaining: 40.4s
9604:	learn: 0.0109377	total: 16m 19s	remaining: 40.3s
9605:	learn: 0.0109364	total: 16m 19s	remaining: 40.2s
9606:	learn:

9738:	learn: 0.0107502	total: 16m 34s	remaining: 26.6s
9739:	learn: 0.0107486	total: 16m 34s	remaining: 26.5s
9740:	learn: 0.0107481	total: 16m 34s	remaining: 26.4s
9741:	learn: 0.0107467	total: 16m 34s	remaining: 26.3s
9742:	learn: 0.0107446	total: 16m 34s	remaining: 26.2s
9743:	learn: 0.0107438	total: 16m 34s	remaining: 26.1s
9744:	learn: 0.0107418	total: 16m 34s	remaining: 26s
9745:	learn: 0.0107405	total: 16m 34s	remaining: 25.9s
9746:	learn: 0.0107395	total: 16m 35s	remaining: 25.8s
9747:	learn: 0.0107380	total: 16m 35s	remaining: 25.7s
9748:	learn: 0.0107356	total: 16m 35s	remaining: 25.6s
9749:	learn: 0.0107340	total: 16m 35s	remaining: 25.5s
9750:	learn: 0.0107329	total: 16m 35s	remaining: 25.4s
9751:	learn: 0.0107309	total: 16m 35s	remaining: 25.3s
9752:	learn: 0.0107300	total: 16m 35s	remaining: 25.2s
9753:	learn: 0.0107283	total: 16m 35s	remaining: 25.1s
9754:	learn: 0.0107267	total: 16m 35s	remaining: 25s
9755:	learn: 0.0107258	total: 16m 35s	remaining: 24.9s
9756:	learn: 0

9888:	learn: 0.0105516	total: 16m 50s	remaining: 11.3s
9889:	learn: 0.0105500	total: 16m 50s	remaining: 11.2s
9890:	learn: 0.0105481	total: 16m 50s	remaining: 11.1s
9891:	learn: 0.0105466	total: 16m 50s	remaining: 11s
9892:	learn: 0.0105458	total: 16m 50s	remaining: 10.9s
9893:	learn: 0.0105440	total: 16m 50s	remaining: 10.8s
9894:	learn: 0.0105427	total: 16m 50s	remaining: 10.7s
9895:	learn: 0.0105412	total: 16m 50s	remaining: 10.6s
9896:	learn: 0.0105397	total: 16m 50s	remaining: 10.5s
9897:	learn: 0.0105375	total: 16m 50s	remaining: 10.4s
9898:	learn: 0.0105364	total: 16m 51s	remaining: 10.3s
9899:	learn: 0.0105357	total: 16m 51s	remaining: 10.2s
9900:	learn: 0.0105355	total: 16m 51s	remaining: 10.1s
9901:	learn: 0.0105346	total: 16m 51s	remaining: 10s
9902:	learn: 0.0105337	total: 16m 51s	remaining: 9.91s
9903:	learn: 0.0105329	total: 16m 51s	remaining: 9.8s
9904:	learn: 0.0105326	total: 16m 51s	remaining: 9.7s
9905:	learn: 0.0105316	total: 16m 51s	remaining: 9.6s
9906:	learn: 0.01

n = 2000

regr = RandomForestRegressor(max_depth=5, n_estimators=n)
regr.fit(data, ys)

dump(regr, '{}regr.joblib'.format(n)) 

print(regr.feature_importances_)

In [11]:
print(model.predict(data[:60]))

[ 1.72660474e-03 -2.34074047e-04  1.15353046e-03 -1.49262534e-03
  3.93981300e-04  1.07424937e-02  1.73010076e-01  1.24436775e-03
 -3.26702968e-03 -6.74289970e-04  1.22603261e-03  2.00729853e-01
  3.60507225e-01  5.42065025e-05  4.48582512e-03 -1.14971663e-03
  2.23087503e-01  4.04464979e-01  5.43101113e-01  2.47993496e-01
  4.41739995e-01  6.02871776e-01  2.21371384e-03  2.41684599e-03
  2.84114644e-01  4.98094220e-01  6.66828762e-01  3.27618390e-01
  5.77008426e-01  7.49934434e-01  3.94303861e-01  6.73007382e-01
  8.55617695e-01  8.01744359e-01  9.88331272e-01  8.11870077e-01
 -2.73101899e-03  5.89676153e-03  3.34623164e-03 -1.29726379e-03
  1.38368480e-03  8.97706853e-04 -5.32756242e-04  6.68277342e-03
 -3.17818138e-03 -2.84686974e-04 -4.53382523e-03 -1.04756014e-04
 -1.05715632e-03  2.03148642e-03 -1.17221254e-03 -4.10838955e-03
  5.92979195e-03  1.10362314e-03  1.89798358e-03  5.07034985e-03
  1.63874077e-03 -8.39242763e-04 -5.30871109e-04 -2.08669057e-03]


In [36]:
with open('answer.txt', 'w') as file:
    pids_, qids_, text_tokens_, question_tokens_, _ = read_csv('dataset.txt', sep='\t')
    for pid, qid, text_token, question_token in zip(pids_, qids_, text_tokens_, question_tokens_):    
        phrases, dt, _ = build_features([(pid, qid, text_token, question_token)])
        if dt:
            ress = model.predict(dt)
        else:
            phrases = [nltk.sent_tokenize(''.join(text_token))[0]]
            ress = [1]
        file.write(qid + "\t" + sorted(zip(ress, phrases), key=lambda x: x[0], reverse=True)[0][1] + '\n')

Text 0 out of 1. Candidates: 0
0.0
Text 0 out of 1. Candidates: 0
0.0
Text 0 out of 1. Candidates: 0
0.0
Text 0 out of 1. Candidates: 0
0.0
Text 0 out of 1. Candidates: 0
0.0
Text 0 out of 1. Candidates: 0
0.0
Text 0 out of 1. Candidates: 0
0.0
Text 0 out of 1. Candidates: 0
0.0
Text 0 out of 1. Candidates: 0
0.0
Text 0 out of 1. Candidates: 0
0.0
Text 0 out of 1. Candidates: 0
0.0
Text 0 out of 1. Candidates: 0
0.0
Text 0 out of 1. Candidates: 0
0.0
Text 0 out of 1. Candidates: 0
0.0
Text 0 out of 1. Candidates: 0
0.0
Text 0 out of 1. Candidates: 0
0.0
Text 0 out of 1. Candidates: 0
0.0
Text 0 out of 1. Candidates: 0
0.0
Text 0 out of 1. Candidates: 0
0.0
Text 0 out of 1. Candidates: 0
0.0
Text 0 out of 1. Candidates: 0
0.0
Text 0 out of 1. Candidates: 0
0.0
Text 0 out of 1. Candidates: 0
0.0
Text 0 out of 1. Candidates: 0
0.0
Text 0 out of 1. Candidates: 0
0.0
Text 0 out of 1. Candidates: 0
0.0
Text 0 out of 1. Candidates: 0
0.0
Text 0 out of 1. Candidates: 0
0.0
Text 0 out of 1. Can

Text 0 out of 1. Candidates: 0
0.0
Text 0 out of 1. Candidates: 0
0.0
Text 0 out of 1. Candidates: 0
0.0
Text 0 out of 1. Candidates: 0
0.0
Text 0 out of 1. Candidates: 0
0.0
Text 0 out of 1. Candidates: 0
0.0
Text 0 out of 1. Candidates: 0
0.0
Text 0 out of 1. Candidates: 0
0.0
Text 0 out of 1. Candidates: 0
0.0
Text 0 out of 1. Candidates: 0
0.0
Text 0 out of 1. Candidates: 0
0.0
Text 0 out of 1. Candidates: 0
0.0
Text 0 out of 1. Candidates: 0
0.0
Text 0 out of 1. Candidates: 0
0.0
Text 0 out of 1. Candidates: 0
0.0
Text 0 out of 1. Candidates: 0
0.0
Text 0 out of 1. Candidates: 0
0.0
Text 0 out of 1. Candidates: 0
0.0
Text 0 out of 1. Candidates: 0
0.0
Text 0 out of 1. Candidates: 0
0.0
Text 0 out of 1. Candidates: 0
0.0
Text 0 out of 1. Candidates: 0
0.0
Text 0 out of 1. Candidates: 0
0.0
Text 0 out of 1. Candidates: 0
0.0
Text 0 out of 1. Candidates: 0
0.0
Text 0 out of 1. Candidates: 0
0.0
Text 0 out of 1. Candidates: 0
0.0
Text 0 out of 1. Candidates: 0
0.0
Text 0 out of 1. Can

Text 0 out of 1. Candidates: 0
0.0
Text 0 out of 1. Candidates: 0
0.0
Text 0 out of 1. Candidates: 0
0.0
Text 0 out of 1. Candidates: 0
0.0
Text 0 out of 1. Candidates: 0
0.0
Text 0 out of 1. Candidates: 0
0.0
Text 0 out of 1. Candidates: 0
0.0
Text 0 out of 1. Candidates: 0
0.0
Text 0 out of 1. Candidates: 0
0.0
Text 0 out of 1. Candidates: 0
0.0
Text 0 out of 1. Candidates: 0
0.0
Text 0 out of 1. Candidates: 0
0.0
Text 0 out of 1. Candidates: 0
0.0
Text 0 out of 1. Candidates: 0
0.0
Text 0 out of 1. Candidates: 0
0.0
Text 0 out of 1. Candidates: 0
0.0
Text 0 out of 1. Candidates: 0
0.0
Text 0 out of 1. Candidates: 0
0.0
Text 0 out of 1. Candidates: 0
0.0
Text 0 out of 1. Candidates: 0
0.0
Text 0 out of 1. Candidates: 0
0.0
Text 0 out of 1. Candidates: 0
0.0
Text 0 out of 1. Candidates: 0
0.0
Text 0 out of 1. Candidates: 0
0.0
Text 0 out of 1. Candidates: 0
0.0
Text 0 out of 1. Candidates: 0
0.0
Text 0 out of 1. Candidates: 0
0.0
Text 0 out of 1. Candidates: 0
0.0
Text 0 out of 1. Can

Text 0 out of 1. Candidates: 0
0.0
Text 0 out of 1. Candidates: 0
0.0
Text 0 out of 1. Candidates: 0
0.0
Text 0 out of 1. Candidates: 0
0.0
Text 0 out of 1. Candidates: 0
0.0
Text 0 out of 1. Candidates: 0
0.0
Text 0 out of 1. Candidates: 0
0.0
Text 0 out of 1. Candidates: 0
0.0
Text 0 out of 1. Candidates: 0
0.0
Text 0 out of 1. Candidates: 0
0.0
Text 0 out of 1. Candidates: 0
0.0
Text 0 out of 1. Candidates: 0
0.0
Text 0 out of 1. Candidates: 0
0.0
Text 0 out of 1. Candidates: 0
0.0
Text 0 out of 1. Candidates: 0
0.0
Text 0 out of 1. Candidates: 0
0.0
Text 0 out of 1. Candidates: 0
0.0
Text 0 out of 1. Candidates: 0
0.0
Text 0 out of 1. Candidates: 0
0.0
Text 0 out of 1. Candidates: 0
0.0
Text 0 out of 1. Candidates: 0
0.0
Text 0 out of 1. Candidates: 0
0.0
Text 0 out of 1. Candidates: 0
0.0
Text 0 out of 1. Candidates: 0
0.0
Text 0 out of 1. Candidates: 0
0.0
Text 0 out of 1. Candidates: 0
0.0
Text 0 out of 1. Candidates: 0
0.0
Text 0 out of 1. Candidates: 0
0.0
Text 0 out of 1. Can

0.0
Text 0 out of 1. Candidates: 0
0.0
Text 0 out of 1. Candidates: 0
0.0
Text 0 out of 1. Candidates: 0
0.0
Text 0 out of 1. Candidates: 0
0.0
Text 0 out of 1. Candidates: 0
0.0
Text 0 out of 1. Candidates: 0
0.0
Text 0 out of 1. Candidates: 0
0.0
Text 0 out of 1. Candidates: 0
0.0
Text 0 out of 1. Candidates: 0
0.0
Text 0 out of 1. Candidates: 0
0.0
Text 0 out of 1. Candidates: 0
0.0
Text 0 out of 1. Candidates: 0
0.0
Text 0 out of 1. Candidates: 0
0.0
Text 0 out of 1. Candidates: 0
0.0
Text 0 out of 1. Candidates: 0
0.0
Text 0 out of 1. Candidates: 0
0.0
Text 0 out of 1. Candidates: 0
0.0
Text 0 out of 1. Candidates: 0
0.0
Text 0 out of 1. Candidates: 0
0.0
Text 0 out of 1. Candidates: 0
0.0
Text 0 out of 1. Candidates: 0
0.0
Text 0 out of 1. Candidates: 0
0.0
Text 0 out of 1. Candidates: 0
0.0
Text 0 out of 1. Candidates: 0
0.0
Text 0 out of 1. Candidates: 0
0.0
Text 0 out of 1. Candidates: 0
0.0
Text 0 out of 1. Candidates: 0
0.0
Text 0 out of 1. Candidates: 0
0.0
Text 0 out of 1.

In [ ]:
answer_direct_in = 0
question_direct_in = 0

total = 0

for text_tokens, question_tokens, answer_tokens in zip(texts, questions, answers):
    text, question, answer = map(lambda x:(''.join(x)), [text_tokens, question_tokens, answer_tokens])
    
    if answer in text:
        #print(text, question, answer, sep = '\n', end='\n\n')
        answer_direct_in += 1
    if ''.join(question_tokens[2:-1]) in text:
        #print(text, question, answer, sep = '\n', end='\n\n')
        question_direct_in += 1
    total += 1

print(answer_direct_in / total, question_direct_in / total)

In [ ]:
stemmer.analyze("привет петя аня что делаешь в древнем египте")

In [ ]:
stemmer.lemmatize("привет петя что делаешь")

In [ ]:
stemmer.analyze("ii")